In [40]:
using CSV, DataFrames, Statistics, Random, Plots, JuMP, Gurobi

In [41]:
using ScikitLearn
@sk_import linear_model: LinearRegression
@sk_import linear_model: Lasso
@sk_import model_selection: train_test_split

┌ Warning: Module model_selection has been ported to Julia - try `import ScikitLearn: CrossValidation` instead
└ @ ScikitLearn.Skcore ~/.julia/packages/ScikitLearn/sqLdT/src/Skcore.jl:259


PyObject <function train_test_split at 0x2ad46dea0>

In [42]:
function opt_split(X, y, n, p, k, lambda, weight = nothing)
    # add column of ones
    X = hcat(ones(Int, size(X, 1)), X)
    p = p + 1

    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)

    @variable(model, theta)
    @variable(model, u[1:n] >= 0)
    @variable(model, beta[1:p])
    @variable(model, w[1:p])

    @objective(model, Min, k * theta + sum(u) + lambda * sum(w))

    for i in 1:p
        @constraint(model, w[i] >= beta[i])
        @constraint(model, w[i] >= -beta[i])
    end 

    for i in 1:n
        if weight == nothing
            @constraint(model, theta + u[i] >= y[i] - sum(X[i, :].*beta))
            @constraint(model, theta + u[i] >= -(y[i] - sum(X[i, :].*beta)))
        else 
            @constraint(model, theta + u[i] >=  weight[i] * (y[i] - sum(X[i, :].*beta)))
            @constraint(model, theta + u[i] >= - weight[i] * (y[i] - sum(X[i, :].*beta)))
        end 
    end
    
    optimize!(model)

    return value(theta), value.(u), value.(beta), value.(w)
end

opt_split (generic function with 2 methods)

In [204]:
function split_train_val(X, y, beta_star, train_fraction = 0.7, weights = nothing)
    # add column of ones
    X = hcat(ones(Int, size(X, 1)), X)

    residuals = y - X * beta_star

    if weights == nothing
        sorted_indices = sortperm(abs.(residuals), rev=true)
    else 
        residuals_weights = [residuals[i] * weights[i] for i in 1:length(weights)]
        sorted_indices = sortperm(abs.(residuals_weights), rev=true)
    end

    num_train_points = round(Int, train_fraction * length(sorted_indices))

    train_indices = sorted_indices[1:num_train_points]

    val_indices = setdiff(1:length(y), train_indices)

    X_train = X[train_indices, 2:end]
    y_train = y[train_indices, :]

    X_val = X[val_indices, 2:end]
    y_val = y[val_indices, :]

    return X_train, y_train, X_val, y_val
end

split_train_val (generic function with 3 methods)

In [205]:
function fit_evaluate_model(X_train, X_val, X_test, y_train, y_val, y_test, lambdas, print = false)
    current_lambda = Inf
    current_val_mse = Inf
    current_test_mse = Inf
    
    for lambda in lambdas

        model = Lasso(alpha=lambda, max_iter = 20000)
        fit!(model, X_train, y_train)
        
        y_pred_val = predict(model, X_val)
        val_mse_i = mse(y_val, y_pred_val)
        
        if current_val_mse >= val_mse_i
            current_lambda = lambda
            current_val_mse = val_mse_i
            current_test_mse = mse(y_test, predict(model, X_test))
        end
    end

    if print == true
        println("best lambda: ", current_lambda)
        println("val score: ", current_val_mse)
        println("test score: ", current_test_mse)
    end

    return current_val_mse, current_test_mse
end

fit_evaluate_model (generic function with 2 methods)

In [206]:
function mse(y_true, y_pred)
    mse = mean((y_true .- y_pred).^2)
    return mse
end

mse (generic function with 1 method)

In [1]:
function normalize_data(X_train, X_test)
    # Calculate mean and standard deviation from training data
    mean_vals = mean(X_train, dims=1)
    std_vals = std(X_train, dims=1)

    # Normalize training data
    X_train_norm = (X_train .-mean_vals) ./ std_vals;
    X_test_norm = (X_test .-mean_vals) ./ std_vals;

    return X_train_norm, X_test_norm
end

normalize_data (generic function with 1 method)

## Single run on different datasets

In [295]:
## hyperparamters which are common to all datasets
train_fraction = 0.7
lambda = 0.001;
lambdas = [0.00001, 0.0001, 0.001, 0.01, 0.1];

### Abalone dataset

In [267]:
# Load the data
df = CSV.read("data/abalone/abalone_original.csv", DataFrame)
selected_columns = setdiff(names(df), ["rings", "sex"])

X_ad = df[:,selected_columns]
y_ad = CSV.read("data/abalone/abalone_original.csv", DataFrame)[!,"rings"];

In [268]:
(train_X, train_y), (test_X, test_y) =
    IAI.split_data(:regression, X_ad, y_ad, train_proportion=0.9, seed = 1);
    
train_X_norm, test_X_norm = normalize_data(Matrix(train_X), Matrix(test_X));

In [291]:
n = nrow(train_X)
p = ncol(train_X)
k = n * train_fraction;

In [297]:
#get optimised split
_, _, betas_star, _ = opt_split(train_X_norm, Array(train_y), n, p, k, lambda)
X_train_opt, y_train_opt, X_val_opt, y_val_opt= split_train_val(train_X_norm, Array(train_y), betas_star, train_fraction);

#get non optimised split 
(X_train_rand, y_train_rand), (X_val_rand, y_val_rand) =
    IAI.split_data(:regression, train_X_norm, train_y, train_proportion=train_fraction, seed = 1);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11


In [296]:
fit_evaluate_model(X_train_opt, X_val_opt, test_X_norm, y_train_opt, y_val_opt, test_y, lambdas, true)

best lambda: 1.0e-5
val score: 8.721706155176252
test score: 94.20870682522202


(8.721706155176252, 94.20870682522202)

In [274]:
fit_evaluate_model(X_train_rand, X_val_rand, test_X_norm, y_train_rand, y_val_rand, test_y, lambdas, true)

best lambda: 0.01
val score: 5.049470331245975
test score: 4.926031381376419


(5.049470331245975, 4.926031381376419)

### Computer Hardware dataset

In [298]:
df = CSV.read("data/comp_hard/machine.data", header = false, DataFrame)
df_names = ["Vendor_Name", "Model_Name", "MYCT", "MMIN", "MMAX", "CACH", "CHMIN", "CHMAX", "PRP", "ERP"]
rename!(df, Symbol.(df_names))

feature_list = ["MYCT", "MMIN", "MMAX", "CACH", "CHMIN", "CHMAX", "PRP"]
X_ch = df[:, feature_list]
y_ch = df[!, "ERP"];

In [299]:
(train_X, train_y), (test_X, test_y) =
    IAI.split_data(:regression, X_ch, y_ch, train_proportion=0.9, seed = 1);

train_X_norm, test_X_norm = normalize_data(Matrix(train_X), Matrix(test_X));

In [300]:
n = nrow(train_X)
p = ncol(train_X)
k = n * train_fraction;

In [301]:
#get optimised split
_, _, betas_star, _ = opt_split(train_X_norm, Array(train_y), n, p, k, lambda)
X_train_opt, y_train_opt, X_val_opt, y_val_opt= split_train_val(train_X_norm, Array(train_y), betas_star, train_fraction);

#get non optimised split 
(X_train_rand, y_train_rand), (X_val_rand, y_val_rand) =
    IAI.split_data(:regression, train_X_norm, train_y, train_proportion=train_fraction, seed = 1);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11


In [302]:
fit_evaluate_model(X_train_opt, X_val_opt, test_X_norm, y_train_opt, y_val_opt, test_y, lambdas, true)

best lambda: 0.1
val score: 154.72881478253643
test score: 822.2747374717602


(154.72881478253643, 822.2747374717602)

In [303]:
fit_evaluate_model(X_train_rand, X_val_rand, test_X_norm, y_train_rand, y_val_rand, test_y, lambdas, true)

best lambda: 0.01
val score: 1122.5642422566282
test score: 966.6884727477184


(1122.5642422566282, 966.6884727477184)

### Concrete dataset

In [304]:
df = CSV.read("data/concrete/concrete_data.csv", DataFrame)
selected_columns = setdiff(names(df), ["concrete_compressive_strength"])

X_c = df[:,selected_columns]
y_c = df[!,"concrete_compressive_strength"];

In [305]:
(train_X, train_y), (test_X, test_y) =
    IAI.split_data(:regression, X_c, y_c, train_proportion=0.9, seed = 2);

train_X_norm, test_X_norm = normalize_data(Matrix(train_X), Matrix(test_X));

In [306]:
#get optimised split 
n = nrow(train_X)
p = ncol(train_X)
k = n * train_fraction


648.9

In [307]:
#get optimised split
_, _, betas_star, _ = opt_split(train_X_norm, Array(train_y), n, p, k, lambda)
X_train_opt, y_train_opt, X_val_opt, y_val_opt= split_train_val(train_X_norm, Array(train_y), betas_star, train_fraction);

#get non optimised split 
(X_train_rand, y_train_rand), (X_val_rand, y_val_rand) =
    IAI.split_data(:regression, train_X_norm, train_y, train_proportion=train_fraction, seed = 1);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11


In [308]:
fit_evaluate_model(X_train_opt, X_val_opt, test_X_norm, y_train_opt, y_val_opt, test_y, lambdas, true)

best lambda: 1.0e-5
val score: 8.721706155176252
test score: 94.20870682522202


(8.721706155176252, 94.20870682522202)

In [309]:
fit_evaluate_model(X_train_rand, X_val_rand, test_X_norm, y_train_rand, y_val_rand, test_y, lambdas, true)

best lambda: 1.0e-5
val score: 107.00856660967708
test score: 97.18099953342731


(107.00856660967708, 97.18099953342731)

## Repeat several times

In [286]:
function repeat_stable_reg(X_full, y_full, num_repeats = 100)
    opt_mse_val_score = []
    opt_mse_test_score = []
    rand_mse_test_score = []
    rand_mse_val_score = []
    lambdas = [0.000001,0.00001, 0.0001, 0.001, 0.01, 0.1]

    lambda_4_opt = 0.001;
    train_fraction = 0.7 #spliting the train into train and val  

    for i in 1:num_repeats

        (train_X, train_y), (test_X, test_y) =
        IAI.split_data(:regression, X_full, y_full, train_proportion=0.9, seed = i);

        train_X_norm, test_X_norm = normalize_data(Matrix(train_X), Matrix(test_X));

        n = nrow(train_X)
        p = ncol(train_X)
        k = n * train_fraction

        #get opt split
        _, _, betas_star, _ = opt_split(train_X_norm, Array(train_y), n, p, k, lambda_4_opt)
        X_train_opt, y_train_opt, X_val_opt, y_val_opt= split_train_val(train_X_norm, Array(train_y), betas_star);

        #get random split 
        (X_train_rand, y_train_rand), (X_val_rand, y_val_rand) = 
            IAI.split_data(:regression, train_X_norm, train_y, train_proportion=train_fraction, seed = 1);

        opt_val_mse, opt_test_mse = fit_evaluate_model(X_train_opt, X_val_opt, test_X_norm, y_train_opt, y_val_opt, test_y, lambdas)
        rand_val_mse, rand_test_mse = fit_evaluate_model(X_train_rand, X_val_rand, test_X_norm, y_train_rand, y_val_rand, test_y, lambdas)
        
        push!(opt_mse_val_score, opt_val_mse)
        push!(opt_mse_test_score, opt_test_mse)

        push!(rand_mse_val_score, rand_val_mse)
        push!(rand_mse_test_score, rand_test_mse)
            
    end

    return opt_mse_val_score, opt_mse_test_score, rand_mse_val_score, rand_mse_test_score

end

repeat_stable_reg (generic function with 2 methods)

In [287]:
function calculate_percentage_improvement(opt_scores, rand_scores)
    if length(opt_scores) != length(rand_scores)
        error("Old scores and new scores must have the same length.")
    end
    
    percentage_improvements = Float64[]

    for i in 1:length(opt_scores)
        opt_score = opt_scores[i]
        rand_score = rand_scores[i]

        percentage_improvement = ((rand_score - opt_score) / rand_score) * 100
        push!(percentage_improvements, percentage_improvement)
    end

    return mean(percentage_improvements)
end

calculate_percentage_improvement (generic function with 1 method)

### 100 runs for concrere dataset

In [289]:
opt_mse_val_score, opt_mse_test_score, rand_mse_val_score, rand_mse_test_score = repeat_stable_reg(X_c, y_c, 100);
meam_percentage_diff = calculate_percentage_improvement(opt_mse_test_score, rand_mse_test_score)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11

0.11553954690504903

### 1000 runs for Computer Hardware dataset

In [140]:
opt_mse_val_score, opt_mse_test_score, rand_mse_val_score, rand_mse_test_score = repeat_stable_reg(X_ch, y_ch, 1000);
meam_percentage_diff = calculate_percentage_improvement(opt_mse_test_score, rand_mse_test_score)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11

2.0272327257000975

### 100 runs for Abalone

In [138]:
opt_mse_val_score, opt_mse_test_score, rand_mse_val_score, rand_mse_test_score = repeat_stable_reg(X_ad, y_ad, 100);
meam_percentage_diff = calculate_percentage_improvement(opt_mse_test_score, rand_mse_test_score)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11

0.05271873630314498